In [65]:
using Revise
using Dates
using Distributions
using Random
using DataFrames
using CSV
using Plots
using StatsBase
using LinearAlgebra
using Statistics
import Pkg
Pkg.activate("..")
Pkg.instantiate()

using SequentialNorta

In [66]:
scen_generator = scenario_generator_init()

Dict{Any, Any} with 29 entries:
  "solar_actual_avg_cf"         => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010627…
  "lp_wind"                     => [0.606061 0.707071 … 0.818182 0.99; 0.888889…
  "wind_marginals_by_issue"     => DataFrame[48×103 DataFrame…
  "load_landing_probabilities"  => [0.888889, 0.585859, 0.545455, 0.505051, 0.2…
  "solar_actual_avg"            => 8760×2 DataFrame…
  "lp_load"                     => [0.888889 0.585859 … 0.939394 0.919192; 0.74…
  "solar_well_defined_cols"     => [7, 8, 9, 10, 11, 12, 13, 14, 15, 16  …  35,…
  "solar_model_data"            => 8736×2 DataFrame…
  "load_marginals_by_issue"     => DataFrame[48×103 DataFrame…
  "unique_forecast_times"       => [DateTime("2018-01-01T00:00:00"), DateTime("…
  "M_load"                      => [1.0 0.0 … 0.0 0.0; 0.860519 0.509419 … 0.0 …
  "solar_landing_probabilities" => [0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0…
  "forecast_scenario_length"    => 48
  "wind_landing_probabilities"  => [0.606061, 0.70

In [67]:
# Save the objects from scen_generator into individual variables
unique_forecast_times = scen_generator["unique_forecast_times"]
unique_issue_times = scen_generator["unique_issue_times"]
start_date = scen_generator["start_date"]
corr_forecast_issue_times = scen_generator["corr_forecast_issue_times"]
forecast_scenario_length = scen_generator["forecast_scenario_length"]
number_of_scenarios = scen_generator["number_of_scenarios"]
solar_model_data = scen_generator["solar_model_data"]
M_load = scen_generator["M_load"]
M_solar = scen_generator["M_solar"]
M_wind = scen_generator["M_wind"]
lp_solar = scen_generator["lp_solar"]
load_marginals_by_issue = scen_generator["load_marginals_by_issue"]
solar_marginals_by_issue = scen_generator["solar_marginals_by_issue"]
wind_marginals_by_issue = scen_generator["wind_marginals_by_issue"]
load_landing_probabilities = scen_generator["load_landing_probabilities"]
solar_landing_probabilities = scen_generator["solar_landing_probabilities"]
wind_landing_probabilities = scen_generator["wind_landing_probabilities"]
load_actual_avg = scen_generator["load_actual_avg"]
solar_actual_avg = scen_generator["solar_actual_avg"]
wind_actual_avg = scen_generator["wind_actual_avg"]
solar_well_defined_cols = scen_generator["solar_well_defined_cols"]
solar_issue_decn_time_matrix = scen_generator["solar_issue_decn_time_matrix"]
load_actual_avg_GW = scen_generator["load_actual_avg_GW"]
solar_actual_avg_cf = scen_generator["solar_actual_avg_cf"]
wind_actual_avg_cf = scen_generator["wind_actual_avg_cf"]
decision_mdl_lkd_length = scen_generator["decision_mdl_lkd_length"];

In [68]:
# Set the date and time for the forecasts
start_index = findfirst(isequal(start_date), unique_forecast_times)

Tstart = start_index;
Tend = decision_mdl_lkd_length - 50; # should be -48 probablys

# Initialize Locked Scenario Path Information
scen_path = Array{Any}(undef, Tend)

Random.seed!(12345);
# intialize normal distribution
normal_dist = Normal(0,1);

In [69]:
# initialize scenario path for load, solar and wind
scen_path_load = Array{Matrix{Float64}}(undef, Tend, number_of_scenarios)
scen_path_solar = Array{Matrix{Float64}}(undef, Tend, number_of_scenarios)
scen_path_wind = Array{Matrix{Float64}}(undef, Tend, number_of_scenarios)

8686×20 Matrix{Matrix{Float64}}:
 #undef  #undef  #undef  #undef  #undef  …  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef  …  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
   ⋮                                     ⋱                          
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #unde

In [70]:
R = range(Tstart, 100, step=1)

1:1:100

In [132]:
# for r in R[1]
    r = 11
    global decision_date = start_date + Dates.Hour(r - 1)

    #=======================================================================
    DEFINE DATETIMES AND FORECAST VS ACTUAL VS MODEL TIMES OR HOURS
    =======================================================================#

    horizon_start_index = findfirst(isequal(decision_date), unique_forecast_times);

    current_issue = corr_forecast_issue_times[horizon_start_index, :issue_time];

    issue_index = findall(x -> x == current_issue, unique_issue_times)[1];
    # find next issue time and compare to start date
    # next_issue = unique_issue_times[issue_index + 1];
    # define active issues (XXX workshop name) as set of forecasts that are available for the hours to use for lookahead
    active_issues = [current_issue];
    # get the indices of the forecasts of the active issue times
    current_forecast_indices = findall(x -> x in active_issues, corr_forecast_issue_times[!,:issue_time]);

    # get the actual forecast times of the current forecast indices
    current_forecast_times = corr_forecast_issue_times[current_forecast_indices, :forecast_time];
    # get the forecast times that are after the start_data
    forecast_times_start_incl = filter(x -> x >= decision_date, current_forecast_times);
    # calculate the length of the forecast times after the start date
    policy_model_length = length(forecast_times_start_incl);
    policy_lookahead_length = policy_model_length - 1; # minus one always for the existing lookahead...

    # extract the actuals from the last 48 - model_forecast_length
    # actuals_length = 48 - policy_model_length + 1;
    policy_actuals_length = forecast_scenario_length - policy_lookahead_length;

    lookahead_decision_hours = collect(policy_actuals_length:forecast_scenario_length);

    load_current_marginals = load_marginals_by_issue[issue_index];
    solar_current_marginals = solar_marginals_by_issue[issue_index];
    wind_current_marginals = wind_marginals_by_issue[issue_index];

    number_of_scen = number_of_scenarios
    scenario_len = forecast_scenario_length
    date = decision_date
    start_ind = horizon_start_index
    active_iss = active_issues
    corr_forc_iss_times = corr_forecast_issue_times
    d_norm = normal_dist
    M = M_solar
    current_marginals = solar_current_marginals
    landing_probabilities = solar_landing_probabilities
    is_solar = true

    ### generator setup objects
    percentile_column_index = startswith.(names(current_marginals), "p_")

    # establish dimension of lower triangular matrix
    dim_M = size(M,1)

    # if the lower triangular size is not the same forecast length, then assign kernel
    if dim_M != scenario_len
        kernel_scenario_len = dim_M
    else
        kernel_scenario_len = scenario_len
    end

    # get the indices of the forecasts of the active issue times
    curr_forec_indices = findall(x -> x in active_iss, corr_forc_iss_times[!,:issue_time])

    # get the actual forecast times of the current forecast indices
    curr_forec_times = corr_forc_iss_times[curr_forec_indices, :forecast_time]

    # get the forecast times that are after the start_data
    forec_times_start_incl = filter(x -> x >= date, curr_forec_times)

    # calculate the length of the forecast times after the start date
    policy_model_len = length(forec_times_start_incl)
    policy_lookahead_len = policy_model_len - 1
    policy_actuals_len = scenario_len - policy_lookahead_len
    first_forec_decision_time_hour = curr_forec_indices[1]
    
    forecast_decision_time_hours = collect(first_forec_decision_time_hour:first_forec_decision_time_hour+scenario_len-1)

    # decision_time_sunny_hours = intersect(solar_issue_decn_time_indices, forecast_decision_time_hours)
    policy_time_sunny_hours = solar_well_defined_cols

    # find the actuals sunny hours before and at the policy start
    kernel_actuals_policy_time_hours = policy_time_sunny_hours[policy_time_sunny_hours .<= policy_actuals_len];
    policy_actuals_kernel_len = length(kernel_actuals_policy_time_hours);
    policy_lookahead_kernel_len = kernel_scenario_len - policy_actuals_kernel_len;


    # initialize the matrix for the normal random variables for the calculation of Z
    W = Matrix{Float64}(undef, number_of_scen, kernel_scenario_len);
    # initialize the vector for the cumulative distribution function on Z
    cdf_Z_48 = zeros(scenario_len,1);

    # initialize the matrix for the scenarios
    Y = Matrix{Float64}(undef, number_of_scen, scenario_len)

    for nscen in 1:number_of_scen
        # nscen = 1
        array_actual_Z = Array{Float64}(undef, kernel_scenario_len)
        array_actual_phi = Array{Float64}(undef, kernel_scenario_len)

        for k in 1:kernel_scenario_len
            # k = 1
            # get the issue time specific index of solar for this lookahead time
            issue_solar_index = solar_well_defined_cols[k]
            # print the actual solar value at this time
            println("actual solar value at time ", issue_solar_index, " is ", solar_model_data[issue_solar_index, :avg_actual])
            # get the marginal distribution for this period in the lookahead
            current_marginals_dist = current_marginals[issue_solar_index, percentile_column_index]
            println("current marginal distribution is ", current_marginals_dist)
            vec_current_marginals_dist = collect(current_marginals_dist)
            ecdf_actuals = ecdf(vec_current_marginals_dist)

            actual_lp_via_md = ecdf_actuals(current_marginals[issue_solar_index,:avg_actual])
            if actual_lp_via_md  == 1
                actual_lp_via_md  = 0.99
            elseif actual_lp_via_md  == 0
                actual_lp_via_md  = 0.01
            end
            actual_Z_via_md = quantile(d_norm, actual_lp_via_md)
            array_actual_Z[k] = actual_Z_via_md
        end

        println("array actual Z is ", array_actual_Z)

        W_from_actuals = inv(M) * array_actual_Z

        # set the historical landing probabilities in the first columns of W
        # W[nscen, 1:policy_actuals_len] = historical_lp; # XXX wrong
        W[nscen, 1:policy_actuals_kernel_len] = W_from_actuals[1:policy_actuals_kernel_len]

        # the rest of the columns are filled with random numbers
        W_upd = rand(d_norm, policy_lookahead_kernel_len)

        # fill the rest of the columns with random numbers
        W[nscen, policy_actuals_kernel_len+1:end] = W_upd

        W_lookahead = W[nscen, :]

        Z = M * W_lookahead

        cdf_Z_base = cdf.(d_norm, Z)

        cdf_Z_48[solar_well_defined_cols] = cdf_Z_base

        for k in 1:scenario_len
            # k = 1
            # marginal_id = within_forecast_sunny_hours[k];
            # current_marginals_dist = current_marginals[marginal_id, 3:101];
            current_marginals_dist = current_marginals[k, percentile_column_index];
            current_actual = current_marginals[k, :avg_actual];
            
            # calculate 
            if k in kernel_actuals_policy_time_hours # replace policy_actuals_len with actual values
                Y[nscen, k] = current_actual;
                println("Y[nscen, k] = ", Y[nscen, k])
            else
                Y[nscen, k] = quantile(current_marginals_dist, cdf_Z_48[k]);
            end
        end
        

    

    end



# end


In [126]:
kernel_actuals_policy_time_hours

5-element Vector{Int64}:
  7
  8
  9
 10
 11

In [128]:
Y[1,r:r+10]

11-element Vector{Float64}:
 372.167962
 413.6092793573537
 283.6516708647981
 324.07790430582645
 464.78492959067796
 260.4058381128092
 449.3299825831862
 124.99748210959908
   0.0
   0.0
   0.0

In [129]:
solar_model_data[r:r+10, :avg_actual]

11-element Vector{Float64}:
 372.167962
 385.1170640416667
 429.0009835416667
 459.20847558333327
 429.1285791666666
 453.4773783333333
 312.1685460416667
  54.33028737500001
   0.014625
   0.0
   0.0

In [134]:
Y[:, r:r+10]

20×11 Matrix{Float64}:
 372.168  453.208  480.045  473.777  …  403.49     57.1276   0.0  0.0  0.0
 372.168  445.86   213.751  343.708     336.424    93.4282   0.0  0.0  0.0
 372.168  420.542  335.11   334.21      101.464   137.992    0.0  0.0  0.0
 372.168  420.873  151.588  224.683      81.2284  101.38     0.0  0.0  0.0
 372.168  443.46   265.039  243.97      465.744   111.87     0.0  0.0  0.0
 372.168  354.999  163.529  210.764  …  185.09      8.20364  0.0  0.0  0.0
 372.168  437.802  301.33   201.864      65.0396   93.1774   0.0  0.0  0.0
 372.168  418.846  287.623  440.525     465.246   164.932    0.0  0.0  0.0
 372.168  407.561  189.819  407.158     426.868    92.1531   0.0  0.0  0.0
 372.168  418.329  311.23   399.355     504.828   151.447    0.0  0.0  0.0
 372.168  302.077  188.693  267.392  …  295.639    58.5149   0.0  0.0  0.0
 372.168  310.513  264.659  388.785     491.873    69.8938   0.0  0.0  0.0
 372.168  355.006  299.211  203.669     185.726   116.929    0.0  0.0  0.0
 3